In [1]:
# univariate multi-step lstm
from math import sqrt
from numpy import split
from numpy import array
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM, Dropout ,RepeatVector,TimeDistributed
from sklearn.preprocessing import MinMaxScaler

In [2]:
# load the new file
dataset = read_csv('./data/bitcoin_data.csv', date_parser = True)
dataset.drop(dataset.columns[0], axis=1,inplace = True)
dataset.head()


,close,open,high,low,Volume BTC,NewDateTime,returns,log_returns,SOPR,fundingRate,googleTrend
0,8431.82,8145.28,8455.00,8142.00,11041.87,2020-01-08 00:00:00,1.035179,0.034574,1.011291,0.01,89
1,8370.92,8431.82,8448.14,8328.89,7355.82,2020-01-08 01:00:00,0.992777,-0.007249,1.011291,0.01,86
2,8278.59,8370.92,8422.00,8256.74,4048.91,2020-01-08 02:00:00,0.988970,-0.011091,1.011291,0.01,75
3,8300.34,8278.59,8344.00,8244.93,3947.38,2020-01-08 03:00:00,1.002627,0.002624,1.011291,0.01,68
4,8344.63,8300.34,8355.87,8288.00,2374.55,2020-01-08 04:00:00,1.005336,0.005322,1.011291,0.01,68


In [3]:
# split into train and test
PDate = '2021-10-08'
train = dataset[dataset['NewDateTime']>= '2021-01-01'].copy()
train = train[train['NewDateTime']< PDate].copy()
test = dataset[dataset['NewDateTime']>= PDate].copy()
# restructure into windows of weekly data

train.drop(train.head(len(train)%168).index,inplace=True)
train = train.drop(['NewDateTime'], axis=1)
test.drop(test.tail(len(test)%168).index,inplace=True)
test = test.drop(['NewDateTime'], axis=1)

scaler = MinMaxScaler()
train = scaler.fit_transform(train)
test = scaler.fit_transform(test)
train


array([[0.24288399, 0.22090652, 0.23350073, ..., 1.        , 0.56113408,
        0.34117647],
       [0.24625908, 0.2429843 , 0.24164591, ..., 1.        , 0.56113408,
        0.42352941],
       [0.25249142, 0.24625922, 0.24816137, ..., 1.        , 0.56113408,
        0.44705882],
       ...,
       [0.70848488, 0.70541637, 0.70541014, ..., 0.50128873, 0.35085647,
        0.57647059],
       [0.69783835, 0.70848472, 0.70940192, ..., 0.50128873, 0.35085647,
        0.57647059],
       [0.69460757, 0.69773036, 0.69018274, ..., 0.50128873, 0.35085647,
        0.57647059]])

In [4]:
train = array(split(train, len(train)/24))
# print(len(test))

test = array(split(test, len(test)/24))

In [5]:
# evaluate model and get scores
n_input = 168


In [6]:

# history is a list of weekly data
# prepare data
# train_x, train_y = to_supervised(train, n_input)
n_out=24
# flatten data
data = train.reshape((train.shape[0]*train.shape[1], train.shape[2]))
X, y = list(), list()
in_start = 0
# step over the entire history one time step at a time
for _ in range(len(data)):
	# define the end of the input sequence
	in_end = in_start + n_input
	out_end = in_end + n_out
	# ensure we have enough data for this instance
	if out_end < len(data):
		X.append(data[in_start:in_end, :])
		y.append(data[in_end:out_end, 0])
	# move along one time step
	in_start += 1
train_x ,train_y =array(X), array(y)



In [7]:
train_x.shape

(6360, 168, 10)

In [8]:
 train_y.shape

(6360, 24)

In [9]:
# define parameters
verbose, epochs, batch_size = 1, 20, 32
n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]
# reshape output into [samples, timesteps, features]
train_y = train_y.reshape((train_y.shape[0], train_y.shape[1], 1))
# define model
# define model
model = Sequential()
model.add(LSTM(200, activation='relu', input_shape=(n_timesteps, n_features)))
model.add(Dropout(0.1))
model.add(RepeatVector(n_outputs))
model.add(LSTM(200, activation='relu', return_sequences=True))
model.add(Dropout(0.1))
model.add(TimeDistributed(Dense(100, activation='relu')))
model.add(TimeDistributed(Dense(1)))

model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 200)               168800    
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
repeat_vector (RepeatVector) (None, 24, 200)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 24, 200)           320800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 24, 200)           0         
_________________________________________________________________
time_distributed (TimeDistri (None, 24, 100)           20100     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 24, 1)             1

In [ ]:
model.compile(loss='mse', optimizer='adam')
# fit network
model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size)

Epoch 1/20
 37/199 [====>.........................] - ETA: 4:11 - loss: 0.0331

In [39]:
history = [x for x in train]
# walk-forward validation over each week
predictions = list()
for i in range(len(test)):
    # predict the week
    data = array(history)
	data = data.reshape((data.shape[0]*data.shape[1], data.shape[2]))
	# retrieve last observations for input data
	input_x = data[-n_input:, :]
	# reshape into [1, n_input, n]
	input_x = input_x.reshape((1, input_x.shape[0], input_x.shape[1]))
	# forecast the next week
	yhat = model.predict(input_x, verbose=0)
	# we only want the vector forecast
	yhat = yhat[0]
    # store the predictions
    predictions.append(yhat)
    # get real observation and add to history for predicting the next week
    history.append(test[i, :])
# evaluate predictions days for each week
predictions = array(predictions)



ValueError: in user code:

    /opt/conda/lib/python3.6/site-packages/keras/engine/training.py:1586 predict_function  *
        return step_function(self, iterator)
    /opt/conda/lib/python3.6/site-packages/keras/engine/training.py:1576 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /opt/conda/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /opt/conda/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /opt/conda/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    /opt/conda/lib/python3.6/site-packages/keras/engine/training.py:1569 run_step  **
        outputs = model.predict_step(data)
    /opt/conda/lib/python3.6/si

In [ ]:
actual = test[:, :, 0]
predicted = predictions
scores = list()
# calculate an RMSE score for each day
for i in range(actual.shape[1]):
	# calculate mse
	mse = mean_squared_error(actual[:, i], predicted[:, i])
	# calculate rmse
	rmse = sqrt(mse)
	# store
	scores.append(rmse)
# calculate overall RMSE
s = 0
for row in range(actual.shape[0]):
	for col in range(actual.shape[1]):
		s += (actual[row, col] - predicted[row, col])**2
score = sqrt(s / (actual.shape[0] * actual.shape[1]))

In [ ]:
# summarize scores
summarize_scores('lstm', score, scores)

In [ ]:
# plot scores
days = ['sun', 'mon', 'tue', 'wed', 'thr', 'fri', 'sat']
pyplot.plot(days, scores, marker='o', label='lstm')
pyplot.show()